In [1]:
import numpy as np
import pandas as pd
import re 
import os 
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Model, Sequential 
from keras.layers import Dense, Embedding, Input, Flatten, Embedding, Dropout
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU , SpatialDropout1D, CuDNNLSTM, TimeDistributed, CuDNNGRU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop, Adam
from keras.engine.topology import Layer, InputSpec
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tensorflow import set_random_seed
from numpy.random import seed
from nltk import tokenize 

from keras.models import Model
from keras import backend as K

from keras.engine.topology import Layer, InputSpec
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.preprocessing import text

from keras.optimizers import RMSprop, Adam
from keras.regularizers import l2
from keras.layers import BatchNormalization, SpatialDropout1D, LSTM
from keras.layers import maximum, GlobalAveragePooling1D

from numpy.random import seed
from tensorflow import set_random_seed
import random as rn
import os

Using TensorFlow backend.


In [2]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#2. Get the file
data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'

#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_sent_punkt.csv')
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_sent_punkt.csv')
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_sent_punkt.csv')

In [0]:
train['mal']         = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [0]:
#words_in_post = train.comment_text.apply(lambda x: len(str(x).split()))
#let_in_post = train.comment_text.apply(lambda x: len(str(x)))

In [0]:
max_features = 40000
maxlen = 400
dropout_rate = 0
rs = 42
epochs = 4
batch_size = 256
embed_dim = 50
rec_units = 150


seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [0]:
if K.backend() == 'tensorflow':
    import tensorflow as tf

    config = tf.ConfigProto(intra_op_parallelism_threads=16,  
                            inter_op_parallelism_threads=16, 
                            allow_soft_placement=True, 
                            device_count = {'CPU': 16})
    session = tf.Session(config=config)
    K.set_session(session)
    
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('zeros')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
class AucPrRecEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = average_precision_score(self.y_val, y_pred)
            print("\n AUC-Precision Recall - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [0]:
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c = 0
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)

In [0]:
import re
sentences_dot = train.comment_text.apply(lambda x: len(x.split('.')))
sentences_q   = train.comment_text.apply(lambda x: len(x.split('?')))
sentences_ex  = train.comment_text.apply(lambda x: len(x.split('!')))

In [0]:
sentences = sentences_dot + sentences_ex + sentences_q

In [30]:
sentences.describe()

count    159571.000000
mean          8.322208
std          27.070768
min           3.000000
25%           5.000000
50%           6.000000
75%           9.000000
max        4945.000000
Name: comment_text, dtype: float64

In [0]:
#max_sen_len = 50
#max_sent_amount = 25
max_sen_len = 40
max_sent_amount = 15

In [39]:
#from tensorflow.python.client import device_lib

#print(device_lib.list_local_devices())
from keras.layers import GlobalAveragePooling1D
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
def clean_str(string):
    string = string.replace(",", ".").replace(";", ".").replace(":", ".").replace("-", ".")
    return string.strip().lower()

def tok_sentence(s):
    #temp = t.texts_to_sequences(s.replace("?", " ").replace("!", " ").replace(".", " ").split())
    temp = tokenizer.texts_to_sequences(s)
    #print(s)
    if len(temp)==0:
        return [[0]]
    #else:
    #    temp = np.concatenate(temp).astype(int)
    #if type(temp[0])==list:
    #    temp = [item for sublist in temp for item in sublist]
    return temp#.reshape(-1)  

padding = True

train_posts = []
train_labels = []
train_texts = []

# FULL TRAIN
for i, value in enumerate(X):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    #train_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = [re.sub(r"[^A-Za-z0-9]", " ", i).strip() for i in sentences]
    sentences = [s for s in sentences if len(s.split())>1]
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent_amount
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent_amount
    if len(sentences)>max_sent_amount:
        sentences = sentences[0:max_sent_amount]
    sentences = pad_sequences(sentences, maxlen=max_sen_len, padding='post')
     
    train_posts.append(sentences)

    
test_posts = []
test_labels = []
test_texts = []
    
    
#Test
for i, value in enumerate(X_test):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    #test_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = [re.sub(r"[^A-Za-z0-9]", " ", i).strip() for i in sentences]
    sentences = [s for s in sentences if len(s.split())>1]
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent_amount
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent_amount
    if len(sentences)>max_sent_amount:
        sentences = sentences[0:max_sent_amount]
    sentences = pad_sequences(sentences, maxlen=max_sen_len, padding='post')
    test_posts.append(sentences)
    
    
X_train = np.array(train_posts)
y_train = np.array(y)
X_test =  np.array(test_posts)
y_test = np.array(y_test)

del train_posts
del test_posts

0


AttributeError: ignored

In [37]:
X_train

array([[[   55,     6,    73, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        ...,
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0]],

       [[  152,     1,    13, ...,     0,     0,     0],
        [  526,     1,    82, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        ...,
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0]],

       [[ 1641,     8,    13, ...,     0,     0,     0],
        [ 1198,     1,  7984, ...,     0,     0,     0],
        [   12,    43,    54, ...,     4,     0,     0],
        ...,
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,    

In [0]:
def make_hat(MAX_SENT_LENGTH = max_sen_len, MAX_SENTS=max_sent_amount, MAX_NB_WORDS=max_features, EMBEDDING_DIM=embed_dim, rec_units=rec_units):
    
    sentence_input     = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    embedded_sequences = Embedding(MAX_NB_WORDS+1, EMBEDDING_DIM, trainable=True)(sentence_input)
    embedded_sequences = SpatialDropout1D(dropout_rate)(embedded_sequences)
    l_lstm             = Bidirectional(CuDNNGRU(rec_units, return_sequences=True, kernel_regularizer=l2(0.1)))(embedded_sequences)
    l_att              = AttentionWithContext()(l_lstm)
    sentEncoder        = Model(sentence_input, l_att)

    review_input   = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
    review_encoder = TimeDistributed(sentEncoder)(review_input)
    l_lstm_sent    = Bidirectional(CuDNNGRU(rec_units, return_sequences=True,  kernel_regularizer=l2(0.1)))(review_encoder)
    l_att_sent     = AttentionWithContext()(l_lstm_sent)
    preds          = Dense(1, activation='sigmoid')(l_att_sent)
    model          = Model(review_input, preds)

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=1, clipnorm=1),
                  metrics=['acc'])
    return model

In [36]:
from sklearn.metrics import average_precision_score, roc_auc_score

X = X_train
y = y_train

c = 0
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
        
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index] 
    
    model = make_hat()
    print(model.summary())
    
    PrAuc = AucPrRecEvaluation(validation_data=(X_val, y_val), interval=1)
    #filepath="weights_base.best.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
    #early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
    callbacks_list = [PrAuc] #[checkpoint, early, PrAuc]
    
    
    print('Fitting')
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=False,
              validation_data=(X_val, y_val),  verbose=1)
    
    probs = model.predict(X_val, batch_size=batch_size, verbose=1)
    
    model.save_weights(f'{cv_models_path}/HAN_fold_{c}.h5')
    
    
    auc_f = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')
    c += 1
    
    del model

 fold 0
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 15, 25)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 300)           2272450   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 15, 300)           406800    
_________________________________________________________________
attention_with_context_2 (At (None, 300)               90600     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 301       
Total params: 2,770,151
Trainable params: 2,770,151
Non-trainable params: 0
_________________________________________________________________
None
Fitting
Instructions for updating:
Use tf.cast 

KeyboardInterrupt: ignored

In [0]:
np.array(auc).mean()

In [0]:
np.array(roc).mean()

In [0]:
def clean_str(string):
    string = string.replace(",", ".").replace(";", ".").replace(":", ".").replace("-", ".")
    return string.strip().lower()

def tok_sentence(s):
    #temp = t.texts_to_sequences(s.replace("?", " ").replace("!", " ").replace(".", " ").split())
    temp = tokenizer.texts_to_sequences(s)
    #print(s)
    if len(temp)==0:
        return np.array([0])
    #else:
    #    temp = np.concatenate(temp).astype(int)
    #if type(temp[0])==list:
    #    temp = [item for sublist in temp for item in sublist]
    return temp#.reshape(-1)  



train_posts = []
train_labels = []
train_texts = []

# FULL TRAIN
for i, value in enumerate(X):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    train_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent_amount
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent_amount

    if len(sentences)>max_sent_amount:
        sentences = sentences[0:max_sent_amount]
    sentences = pad_sequences(sentences, maxlen=max_sen_len)

    train_posts.append(sentences)

    
test_posts = []
test_labels = []
test_texts = []
    
    
#Test
for i, value in enumerate(X_test):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    test_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent_amount
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent_amount

    if len(sentences)>max_sent_amount:
        sentences = sentences[0:max_sent_amount]
    sentences = pad_sequences(sentences, maxlen=max_sen_len)

    test_posts.append(sentences)
    
    
X_train = np.array(train_posts)
y_train = np.array(y)
X_test =  np.array(test_posts)
y_test = np.array(y_test)

del train_posts
del test_posts

In [0]:
#X_train, y_train = X, y

model = make_hat()
PrAuc = AucPrRecEvaluation(validation_data=(X_test, y_test), interval=1)
    #filepath="weights_base.best.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
    #early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
callbacks_list = [PrAuc] #[checkpoint, early, PrAuc]

print('Fitting')
model.fit(X_train, y_train, batch_size=256, epochs=epochs, shuffle=False,  validation_data=(X_test, y_test), callbacks = callbacks_list, verbose=1)
#probs = model.predict(X_test, batch_size=600, verbose=1)
from sklearn.metrics import average_precision_score, roc_auc_score
#auc_f = average_precision_score(y_test, probs)
#roc_f = roc_auc_score(y_test, probs)
#print(auc_f)
#print(roc_f)

In [0]:
model.fit(X_train, y_train, batch_size=512, epochs=4, shuffle=False,  validation_data=(X_test, y_test), callbacks = callbacks_list, verbose=1)
#probs = model.predict(X_test, batch_size=batch_size, verbose=1)

Train on 158719 samples, validate on 64830 samples
Epoch 1/4
158719/158719 [==============================] - 524s 3ms/step - loss: 0.1056 - acc: 0.9680 - val_loss: 0.1720 - val_acc: 0.9474

 AUC-Precision Recall - epoch: 1 - score: 0.810497 

Epoch 2/4
158719/158719 [==============================] - 524s 3ms/step - loss: 0.1018 - acc: 0.9695 - val_loss: 0.1749 - val_acc: 0.9475

 AUC-Precision Recall - epoch: 2 - score: 0.807299 

Epoch 3/4
158719/158719 [==============================] - 523s 3ms/step - loss: 0.0987 - acc: 0.9710 - val_loss: 0.1746 - val_acc: 0.9476

 AUC-Precision Recall - epoch: 3 - score: 0.804978 

Epoch 4/4
158719/158719 [==============================] - 524s 3ms/step - loss: 0.0957 - acc: 0.9725 - val_loss: 0.1714 - val_acc: 0.9480

 AUC-Precision Recall - epoch: 4 - score: 0.801119 

